In [7]:
import requests
import json
import time
import Util as util
import pandas as pd
import _thread

with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version",
        "Screensize"
    ]
africa_bound_box = {"MinLatitude": -47.131349, "MaxLatitude": 37.535900,
                    "MinLongitude": -25.383911, "MaxLongitude": 63.808594 }

new_probe_body = {"criteria":{"FromXMinutes": 15, "Sources":[{"BoundingBox":africa_bound_box}],
                    "ProbeInfoProperties":["Latitude","Longitude","ProbeID","Platform","CountryCode","CityName"]
                             }
       }

get_probes = util.api_url + 'GetNewProbes'

In [ ]:
def getTestSettings(cc, videoID, pw, ph, res):
    json_test = {
      "testSettings": {
        "VideoUrl": "http://pcsucdn.com/YoutubeVideoTest2.html?v=" + videoID + '&pw=' + pw + "&ph=" + ph + "&pq=" + res ,
        "VideoControlScript": "Youtube4.js",
        "TestCount": 10,
        "Sources": [
          {
            "CountryCode": cc,
            "Platform" : 'PC'
          }
        ],
        "ProbeInfoProperties": probeInfoProperties,
        "TestResultProperties": [
            {
              "VideoID",
              "VideoResolution",
              "VideoHostname",
              "InitialBufferingLatency",
              "RebufferingTimes",
              "DurationOfEachRebuffering",
              "VideoPlayDuration",
              "VideoDownloadSpeed",
              "VideoPlayFinishEvent"
            }
        ]
      }
    }
    
    return json_test

In [14]:
videos = ['MQ7C-gO4QKA','76EErDmwrUY']
vdo_test_ep= util.api_url + 'StartVideoTest'
df = pd.DataFrame()
probe_info_properties = ["ASN", "IPAddress", "Network", "ConnectionType", "Platform", "Screensize", "DNSResolver",
                         "Latitude", "Longitude", "ProbeID", "CountryCode", "CityName", "NetworkID", 'Screensize' ]

# check probe availability in a country 
def is_probe_avil(cc):
    new_probe_body_cc = {"criteria":{"FromXMinutes": 15,"Sources":[{"CountryCode":cc}],
                        "ProbeInfoProperties":["ProbeID","Platform"]}
           }
    probe_request = requests.post(get_probes, headers=HEADERS, data = json.dumps(new_probe_body_cc))
    probe_result = probe_request.json()
    num_probe = util.check_probe_in_cc(probe_result)
    
    return(num_probe)
    

def run_video_expr(cc):
    col = [
        'test_status', 'test_date', 'DurationOfEachRebuffering', 'VideoID', 'VideoPlayDuration',
        'VideoPlayFinishEvent', 'VideoResolution', 'InitialBufferingStartupLatency','RebufferingTimes',
        'VideoDownloadSpeed', 'ASN','CityName', 'ConnectionType', 'CountryCode', 'DNSResolver', 'IPAddress',
        'Latitude', 'Longitude', 'Network', 'NetworkID', 'Platform', 'ProbeID' 
    ]
    num_probes = is_probe_avil(cc)
    print(num_probes)
    
    if num_probes < 1:
        print('No probe available')
        sys.exit()
    for video in videos:
        video_test_body = {
          "testSettings": {
            "VideoUrl": "http://pcsucdn.com/YoutubeVideoTest.html?v=" + video,
            "VideoControlScript": "Youtube4.js",
            "TestCount": 5,
            "Sources": [
              {
                "CountryCode": cc
              }
            ],
            "ProbeInfoProperties": probe_info_properties,
            "TestResultProperties": [
              "VideoDownloadSpeed"
            ]
          }
        }

        vdo_request = requests.post(vdo_test_ep, headers=HEADERS, data = json.dumps(video_test_body))
        vdo_request_result = vdo_request.json()
        status = vdo_request_result['StartVideoTestResult']['Status']['StatusCode']
        test_id = vdo_request_result['StartVideoTestResult']['TestID']

        # wait for 20 seconds to retrieve the results
        time.sleep(120)
        if int(status) != 200:
            print("there is something wrong in this test, and continue to the next ...")
            continue
        vdo_result = util.retrieve_video_result(test_id)

        results_lst = util.result_to_csv(vdo_result)
        _df = pd.DataFrame.from_dict(results_lst)
        _df = _df[col]
        _df.to_csv('data/africa-youtube.csv', sep=',', mode='a', 
                   header = False, index=False, encoding = "ISO-8859-1")

In [ ]:
# Define a function for the thread
def print_time( threadName, delay):
    count = 0
    while count < 1:
        time.sleep(delay)
        count += 1
        print ("%s: %s" % ( threadName, time.ctime(time.time()) ))

for cc in util.ccs_video.keys():
    try:
        _thread.start_new_thread(run_vide_expr, (cc,))
    except Exception as e:
        print("Error: unable to start thread")
#         print(e)


In [15]:
run_video_expr('ZA')

12
there is something wrong in this test, and continue to the next ...
there is something wrong in this test, and continue to the next ...


In [ ]:
new_probe_body_cc = {"criteria":{"FromXMinutes": 15,"Sources":[{"CountryCode":'UK'}],
                        "ProbeInfoProperties":["ProbeID","Platform","CountryCode"]}
           }
probe_request = requests.post(get_probes, headers=headers, data = json.dumps(new_probe_body_cc))
probe_result = probe_request.json()
num_probe = util.check_probe_in_cc(probe_result)

In [ ]:
num_probe